#  Scrapping from MyCareersFuture.sg  
This code uses Selenium to load the pages as mycareersfuture.sg makes extensive use of javascript to generate dynamic html.

In [1]:
#importing libraries for scraping
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import os
from selenium import webdriver
import requests
import numpy as np
from time import sleep

In [2]:
#Initializing chromedriver
chromedriver = './chromedriver/chromedriver.exe'
os.environ["webdriver.chrome.driver"] = chromedriver

### Defining useful functions and lists

In [3]:
#This function extracts just the links of the job search result (1 page).
def getlinks(soup):
    table = soup.find('div',{'class':'card-list'})
    try:
        table.find('section',{'aria-label':'Suggested Articles'}).extract() 
        #Extracting ads
        #Because ads are annoying and also comes with the href attribute.
    except:
        pass
    
    links = []
    for a in table.findAll('a'):
        links.append(a['href'])
        
    return links

In [4]:
#This function grabs the information from the job posting page.
def onejob(link):
    #link must be a string
    joburl = 'https://www.mycareersfuture.sg'+str(link)

    #open job page with a sub-webdriver.
    subdriver = webdriver.Chrome(chromedriver) 
    subdriver.get(joburl)

    #IMPT: Wait for page to load.
    sleep(6)

    #Getting page source
    jobsoup = BeautifulSoup(subdriver.page_source, 'html.parser')

    #Sub-webdriver's job is done here
    subdriver.close()

    #Getting our variables

    info = jobsoup.find('div',{'class':"jobInfo w-100 dib v-top relative"})
    #info contains all the job information in the header.
    try:
        company = info.find('p',{'name':'company'}).text
    except:
        company = np.nan
    title = info.find('h1',{'id':'job_title'}).text
    try:
        address = info.find('p',{'id':'address'}).text
    except:
        address = np.nan
    try:
        jobtype = info.find('p',{'id':'employment_type'}).text
    except:
        jobtype = np.nan
    try:
        seniority = info.find('p',{'id':'seniority'}).text
    except:
        seniority = np.nan
    try:
        categories = info.find('p',{'id':'job-categories'}).text
    except:
        categories = np.nan

    salary = info.find('div',{'class':'salary tr-l'})
    try:
        min_sal = salary.findAll('span',{'class':'dib'})[2].text
    except:
        min_sal = np.nan
    try:
        max_sal = salary.findAll('span',{'class':'dib'})[3].text.strip('to')
    except:
        max_sal = np.nan
    try:
        sal_type = salary.findAll('span',{'class':'dib'})[4].text
    except:
        sal_type = np.nan
    #description is in a separate div tag.
    description = jobsoup.find('div',{'class':"jobDescription w-100 v-top relative"}).text

    return {'job_title':title, \
            'company':company, \
            'address':address, \
            'job_type':jobtype, \
            'seniority':seniority, \
            'categories':categories, \
            'salary_from':min_sal, \
            'salary_to':max_sal, \
            'salary_type':sal_type, \
            'description':description, \
            'url':joburl}

### Scraping the data onto a DataFrame

In [7]:
#Function for looping through the roles specified above. Takes in a dictionary as defined in the next cell.
def scrapeit(role_dict):
    df = pd.DataFrame(columns = ['job_title','company','address','job_type',\
                                 'seniority','categories','salary_from','salary_to',\
                                 'salary_type', 'description','url'])
    rolecount = 0
    for role,value in role_dict.items(): 
        print('Scrapping',role,'roles.')
        url = value[0]
        pages = value[1]

        #Looping through search result pages. 
        pagecount = 0
        for page in pages:
            #Open page with webdriver
            driver = webdriver.Chrome(chromedriver) 
            driver.get(url+str(page))

            #IMPT: Wait for page to load.
            sleep(5)

            #Parsing page into Beautiful Soup.
            resultsoup = BeautifulSoup(driver.page_source, 'html.parser')

            #driver's job is done here.
            driver.close()

            #Get links of job postings
            links = getlinks(resultsoup)

            #Add jobs to DataFrame
            jobcount = 0
            for link in links:
                #the following for loops does a retry 3 times. If it fails 3 times, we move on to the next job
                #this avoids the whole process from stopping because one job posting didn't load in 5 seconds.
                for attempt in range(3):
                    try:
                        df = df.append(onejob(link), ignore_index=True)
                        jobcount+=1
                        break
                    except:
                        print('Job',jobcount+1,'of page',pagecount+1,'failed. Retrying...')
                else:
                    print('Job',jobcount+1,'of page',pagecount+1,'failed 3 times. Moving on to next job.')
            pagecount+=1
            print(jobcount,'jobs on page',pagecount,'successful.')
        rolecount+=1
        print(rolecount,'roles complete.', role,'role complete.')
    print(len(df),'entries successfully scrapped.')
    return df

In [5]:
df = pd.DataFrame(columns = ['job_title','company','address','job_type','seniority','categories','salary_from','salary_to', 'salary_type', 'description','url'])
df

,job_title,company,address,job_type,seniority,categories,salary_from,salary_to,salary_type,description,url


In [22]:
datascience = 'https://www.mycareersfuture.sg/search?search=data%20science&sortBy=new_posting_date&page='
dspage = range(6)
dataanalyst = 'https://www.mycareersfuture.sg/search?search=data%20analyst&sortBy=new_posting_date&page='
dapage = range(6)
dataengineer = 'https://www.mycareersfuture.sg/search?search=data%20engineer&sortBy=new_posting_date&page='
depage = range(6)
dataarchitect = 'https://www.mycareersfuture.sg/search?search=data%20architect&sortBy=new_posting_date&page='
darchpage = range(2)
machinelearning = 'https://www.mycareersfuture.sg/search?search=machine%20learning&sortBy=new_posting_date&page='
mlpage = range(15)
AI = 'https://www.mycareersfuture.sg/search?search=artificial%20intelligence&sortBy=new_posting_date&page='
aipage = range(6)
roles = {'Data Science':[datascience,dspage], \
          'Data Analyst':[dataanalyst,dapage], \
          'Data Engineer':[dataengineer,depage], \
          'Data Architect':[dataarchitect,darchpage], \
          'Machine Learning':[machinelearning,mlpage], \
          'Artificial Intelligence':[AI,aipage]}

In [ ]:
#Running everything together
df = scrapeit(roles)

### Below code for testing/adding extra roles

In [102]:
bigdata = 'https://www.mycareersfuture.sg/search?search=big%20data&sortBy=new_posting_date&page='
bdpage = range(16)
deeplearning = 'https://www.mycareersfuture.sg/search?search=deep%20learning&sortBy=new_posting_date&page='
dlpage = range(5)
businessdata = 'https://www.mycareersfuture.sg/search?search=business%20data&sortBy=new_posting_date&page='
bsdpage = range(4)
businessintelligence = 'https://www.mycareersfuture.sg/search?search=business%20intelligence&sortBy=new_posting_date&page='
bipage = range(7)
python = 'https://www.mycareersfuture.sg/search?search=python&sortBy=new_posting_date&page='
pypage = range(10)
nlp = 'https://www.mycareersfuture.sg/search?search=natural%20language%20processing&sortBy=new_posting_date&page='
nlpage = range(2)
dataanalytics = 'https://www.mycareersfuture.sg/search?search=data%20analytics&sortBy=new_posting_date&page='
daticspage = range(19)

roles2 ={'Data Analytics':[dataanalytics,daticspage]}

In [103]:
df2 = scrapeit(roles2)

Scrapping Data Analytics roles.
20 jobs on page 1 successful.
20 jobs on page 2 successful.
20 jobs on page 3 successful.
20 jobs on page 4 successful.
20 jobs on page 5 successful.
20 jobs on page 6 successful.
20 jobs on page 7 successful.
20 jobs on page 8 successful.
20 jobs on page 9 successful.
20 jobs on page 10 successful.
20 jobs on page 11 successful.
20 jobs on page 12 successful.
20 jobs on page 13 successful.
20 jobs on page 14 successful.
20 jobs on page 15 successful.
20 jobs on page 16 successful.
20 jobs on page 17 successful.
20 jobs on page 18 successful.
19 jobs on page 19 successful.
1 roles complete. Data Analytics role complete.
379 entries successfully scrapped.


In [136]:
#The non-function way. So that jobs get appended one by one and data isn't lost if an error pops up.
'''
rolecount = 0
for role,value in roles2.items(): 
    print('Scrapping',role,'roles.')
    url = value[0]
    pages = value[1]
    
    #Looping through search result pages. Careersfuture gives 6 pages per search term.
    pagecount = 0
    for page in pages:
        #Open page with webdriver
        driver = webdriver.Chrome(chromedriver) 
        driver.get(url+str(page))

        #IMPT: Wait for page to load.
        sleep(4)

        #Parsing page into Beautiful Soup.
        resultsoup = BeautifulSoup(driver.page_source, 'html.parser')

        #driver's job is done here.
        driver.close()

        #Get links of job postings
        links = getlinks(resultsoup)

        #Add jobs to DataFrame
        jobcount = 0
        for link in links:
            df2 = df2.append(onejob(link), ignore_index=True)
            jobcount+=1
        pagecount+=1
        print(jobcount,'jobs on page',pagecount,'successful.')
    rolecount+=1
    print(rolecount,'roles complete.', role,'role complete.')
print(len(df),'entries successfully scrapped.')

Scrapping Artificial Intelligence roles.
20 jobs on page 1 successful.
20 jobs on page 2 successful.
20 jobs on page 3 successful.
20 jobs on page 4 successful.
20 jobs on page 5 successful.
8 jobs on page 6 successful.
1 roles complete. Artificial Intelligence role complete.
404 entries successfully scrapped.


In [104]:
df

,job_title,company,address,job_type,seniority,categories,salary_from,salary_to,salary_type,description,url
0,Business Development Manager,KKT TECHNOLOGY PTE. LTD.,71 AYER RAJAH CRESCENT 139951,Full Time,"Manager, Professional, Senior Executive","Healthcare / Pharmaceutical, General Management","$4,000","$7,000",Monthly,"Roles & ResponsibilitiesAt Holmusk, our missio...",https://www.mycareersfuture.sg/job/business-de...
1,SIS Faculty - Assistant Professor of Informati...,SINGAPORE MANAGEMENT UNIVERSITY,81 VICTORIA STREET 188065,Full Time,Professional,Education and Training,"$9,000","$15,000",Monthly,Roles & ResponsibilitiesOUR CORE RESEARCH AREA...,https://www.mycareersfuture.sg/job/sis-faculty...
2,Delivery Lead – Singapore (PACE Analytics),RIO TINTO SINGAPORE HOLDINGS PTE. LTD.,NaN,Permanent,Manager,Information Technology,"$13,000","$17,000",Monthly,Roles & ResponsibilitiesDelivery Lead – Singap...,https://www.mycareersfuture.sg/job/delivery-le...
3,IT Scrum Master,Company Undisclosed,NaN,Full Time,Professional,Information Technology,"$7,000","$12,000",Monthly,Roles & Responsibilities There will be a few p...,https://www.mycareersfuture.sg/job/scrum-maste...
4,"Data Science Manager, APAC",AIRBNB SINGAPORE PRIVATE LIMITED,158 CECIL STREET 069545,Full Time,Manager,Sciences / Laboratory / R&D,"$11,000","$20,000",Monthly,Roles & ResponsibilitiesAirbnb is a global pla...,https://www.mycareersfuture.sg/job/data-scienc...
5,"SVP / VP, Lead Data Scientist, Group Consumer ...",DBS BANK LTD.,NaN,"Permanent, Full Time",Senior Management,Banking and Finance,"$9,500","$18,000",Monthly,Roles & ResponsibilitiesJob Purpose The Lead D...,https://www.mycareersfuture.sg/job/svp-vp-lead...
6,Lecturer - IT Applications Development,Institute of Technical Education,2 ANG MO KIO DRIVE 567720,Contract,Executive,"Education and Training, Public / Civil Service","$3,000","$6,000",Monthly,Roles & ResponsibilitiesResponsibilities As a ...,https://www.mycareersfuture.sg/job/lecturer-ap...
7,APAC Data Engineer,UPS ASIA GROUP PTE. LTD.,"UPS HOUSE, 22 CHANGI SOUTH AVENUE 2 486064",Permanent,Senior Executive,Logistics / Supply Chain,"$5,256","$7,008",Monthly,Roles & ResponsibilitiesSummary The Data Engin...,https://www.mycareersfuture.sg/job/apac-data-e...
8,"AVP / Senior Associate, Data Engineer, IBG D...",DBS BANK LTD.,NaN,"Permanent, Full Time","Manager, Senior Executive",Banking and Finance,"$5,500","$11,000",Monthly,Roles & ResponsibilitiesJob Purpose The Data ...,https://www.mycareersfuture.sg/job/avp-senior-...
9,"SVP / VP, Senior Data Scientist, Data & Transf...",DBS BANK LTD.,NaN,"Permanent, Full Time","Senior Management, Middle Management",Banking and Finance,"$9,500","$19,000",Monthly,Roles & Responsibilities Drive enterprise stra...,https://www.mycareersfuture.sg/job/svp-vp-seni...


In [105]:
df2

,job_title,company,address,job_type,seniority,categories,salary_from,salary_to,salary_type,description,url
0,Data Analytics Lead,TRAVELOKA SERVICES PTE. LTD.,"ROBINSON 77, 77 ROBINSON ROAD 068896",Permanent,Manager,Information Technology,"$8,300","$15,000",Monthly,Roles & ResponsibilitiesYou will be responsibl...,https://www.mycareersfuture.sg/job/data-analyt...
1,"SVP, Infrastructure Lead, Group Consumer Banki...",DBS BANK LTD.,"MARINA BAY FINANCIAL CENTRE, 12 MARINA BOULEVA...",Full Time,Senior Management,Information Technology,"$12,000","$21,600",Monthly,Roles & ResponsibilitiesBusiness Function ...,https://www.mycareersfuture.sg/job/svp-infrast...
2,Head (Data Architect),Accountant-General's Department,NaN,Full Time,NaN,"Accounting / Auditing / Taxation, Public / Civ...",,NaN,NaN,Roles & ResponsibilitiesHead (Data Architect)\...,https://www.mycareersfuture.sg/job/head-accoun...
3,Data Analyst - Data Transformation IMO,ROBERT WALTERS (SINGAPORE) PTE LTD,6 BATTERY ROAD 049909,"Temporary, Contract",Executive,Information Technology,"$6,000","$7,400",Monthly,Roles & ResponsibilitiesA new Data Analyst - D...,https://www.mycareersfuture.sg/job/data-analys...
4,"Data Manager, Data Management",AIA SINGAPORE PRIVATE LIMITED,"AIA TOWER, 1 ROBINSON ROAD 048542",Full Time,Senior Management,Information Technology,"$6,000","$12,000",Monthly,Roles & Responsibilities Drive the overall dat...,https://www.mycareersfuture.sg/job/data-manage...
5,"Senior Assistant Director, Survey Design and A...",Ministry of Manpower,18 HAVELOCK ROAD 059764,Full Time,Manager,Public / Civil Service,,NaN,NaN,Roles & ResponsibilitiesRole Purpose\nTo desig...,https://www.mycareersfuture.sg/job/senior-assi...
6,Research Fellow (Biochemistry),NATIONAL UNIVERSITY OF SINGAPORE,21 LOWER KENT RIDGE ROAD 119077,"Contract, Full Time",Professional,Sciences / Laboratory / R&D,"$4,000","$6,000",Monthly,Roles & ResponsibilitiesThe National Universit...,https://www.mycareersfuture.sg/job/research-fe...
7,[m]PLATFORM | Head of Biddable Search APAC,GROUPM ASIA PACIFIC HOLDINGS PTE. LTD.,"CHINA SQUARE CENTRAL, 18 CROSS STREET 048423",Full Time,Middle Management,Advertising / Media,"$12,000","$18,000",Monthly,Roles & ResponsibilitiesWe are looking for a H...,https://www.mycareersfuture.sg/job/mplatform-h...
8,Business Development Director,KKT TECHNOLOGY PTE. LTD.,71 AYER RAJAH CRESCENT 139951,Full Time,"Middle Management, Manager, Senior Executive","Healthcare / Pharmaceutical, General Management","$5,200","$8,000",Monthly,"Roles & ResponsibilitiesAt Holmusk, our missio...",https://www.mycareersfuture.sg/job/business-de...
9,Senior Risk Analyst,PAYPAL PTE. LTD.,"SUNTEC TOWER FIVE, 5 TEMASEK BOULEVARD 038985",Full Time,Professional,Banking and Finance,"$6,000","$12,000",Monthly,Roles & ResponsibilitiesFueled by a fundamenta...,https://www.mycareersfuture.sg/job/senior-risk...


In [106]:
#Appending new data to previous dataset. Also dropping duplicate entries.
df = df.append(df2, ignore_index=True)
df = df.drop_duplicates().reset_index(drop=True)

In [107]:
df

,job_title,company,address,job_type,seniority,categories,salary_from,salary_to,salary_type,description,url
0,Business Development Manager,KKT TECHNOLOGY PTE. LTD.,71 AYER RAJAH CRESCENT 139951,Full Time,"Manager, Professional, Senior Executive","Healthcare / Pharmaceutical, General Management","$4,000","$7,000",Monthly,"Roles & ResponsibilitiesAt Holmusk, our missio...",https://www.mycareersfuture.sg/job/business-de...
1,SIS Faculty - Assistant Professor of Informati...,SINGAPORE MANAGEMENT UNIVERSITY,81 VICTORIA STREET 188065,Full Time,Professional,Education and Training,"$9,000","$15,000",Monthly,Roles & ResponsibilitiesOUR CORE RESEARCH AREA...,https://www.mycareersfuture.sg/job/sis-faculty...
2,Delivery Lead – Singapore (PACE Analytics),RIO TINTO SINGAPORE HOLDINGS PTE. LTD.,NaN,Permanent,Manager,Information Technology,"$13,000","$17,000",Monthly,Roles & ResponsibilitiesDelivery Lead – Singap...,https://www.mycareersfuture.sg/job/delivery-le...
3,IT Scrum Master,Company Undisclosed,NaN,Full Time,Professional,Information Technology,"$7,000","$12,000",Monthly,Roles & Responsibilities There will be a few p...,https://www.mycareersfuture.sg/job/scrum-maste...
4,"Data Science Manager, APAC",AIRBNB SINGAPORE PRIVATE LIMITED,158 CECIL STREET 069545,Full Time,Manager,Sciences / Laboratory / R&D,"$11,000","$20,000",Monthly,Roles & ResponsibilitiesAirbnb is a global pla...,https://www.mycareersfuture.sg/job/data-scienc...
5,"SVP / VP, Lead Data Scientist, Group Consumer ...",DBS BANK LTD.,NaN,"Permanent, Full Time",Senior Management,Banking and Finance,"$9,500","$18,000",Monthly,Roles & ResponsibilitiesJob Purpose The Lead D...,https://www.mycareersfuture.sg/job/svp-vp-lead...
6,Lecturer - IT Applications Development,Institute of Technical Education,2 ANG MO KIO DRIVE 567720,Contract,Executive,"Education and Training, Public / Civil Service","$3,000","$6,000",Monthly,Roles & ResponsibilitiesResponsibilities As a ...,https://www.mycareersfuture.sg/job/lecturer-ap...
7,APAC Data Engineer,UPS ASIA GROUP PTE. LTD.,"UPS HOUSE, 22 CHANGI SOUTH AVENUE 2 486064",Permanent,Senior Executive,Logistics / Supply Chain,"$5,256","$7,008",Monthly,Roles & ResponsibilitiesSummary The Data Engin...,https://www.mycareersfuture.sg/job/apac-data-e...
8,"AVP / Senior Associate, Data Engineer, IBG D...",DBS BANK LTD.,NaN,"Permanent, Full Time","Manager, Senior Executive",Banking and Finance,"$5,500","$11,000",Monthly,Roles & ResponsibilitiesJob Purpose The Data ...,https://www.mycareersfuture.sg/job/avp-senior-...
9,"SVP / VP, Senior Data Scientist, Data & Transf...",DBS BANK LTD.,NaN,"Permanent, Full Time","Senior Management, Middle Management",Banking and Finance,"$9,500","$19,000",Monthly,Roles & Responsibilities Drive enterprise stra...,https://www.mycareersfuture.sg/job/svp-vp-seni...


In [108]:
df.to_csv('./careersfuture.csv', index = False)

In [110]:
#Making sure csv is saved correctly.
new = pd.read_csv('./careersfuture.csv')
new.sample(10)

,job_title,company,address,job_type,seniority,categories,salary_from,salary_to,salary_type,description,url
2,Delivery Lead – Singapore (PACE Analytics),RIO TINTO SINGAPORE HOLDINGS PTE. LTD.,NaN,Permanent,Manager,Information Technology,"$13,000","$17,000",Monthly,Roles & ResponsibilitiesDelivery Lead – Singap...,https://www.mycareersfuture.sg/job/delivery-le...
254,Mid Level Data Engineer,TRAVELOKA SERVICES PTE. LTD.,"ROBINSON 77, 77 ROBINSON ROAD 068896",Permanent,Executive,Information Technology,"$4,300","$7,600",Monthly,Roles & ResponsibilitiesData Engineers at Trav...,https://www.mycareersfuture.sg/job/mid-level-d...
110,Data Analyst,HYDROGEN CONSULTING SOLUTIONS PTE. LTD.,"PRUDENTIAL TOWER, 30 CECIL STREET 049712",Contract,Professional,Banking and Finance,"$4,500","$6,000",Monthly,Roles & Responsibilities· Min. 2 years...,https://www.mycareersfuture.sg/job/data-analys...
260,Big Data Engineer (287102),SIEMENS PTE. LTD.,"THE SIEMENS CENTRE, 60 MACPHERSON ROAD 348615","Permanent, Full Time","Professional, Senior Executive",Information Technology,"$4,000","$7,000",Monthly,Roles & ResponsibilitiesWhat are my responsibi...,https://www.mycareersfuture.sg/job/big-data-en...
441,Quantitative Analyst,NIKKO ASSET MANAGEMENT ASIA LIMITED,NaN,Permanent,Fresh/entry level,Banking and Finance,"$4,000","$4,800",Monthly,Roles & Responsibilities Provide quantitativ...,https://www.mycareersfuture.sg/job/quantitativ...
308,Data Scientist,CAPGEMINI SINGAPORE PTE. LTD.,6 BATTERY ROAD 049909,"Permanent, Contract",Professional,Information Technology,"$9,000","$12,000",Monthly,Roles & Responsibilities Primary Responsibilit...,https://www.mycareersfuture.sg/job/data-scient...
1013,"IT Director, Data & Analytics PLO ASPAC",JOHNSON & JOHNSON PTE. LTD.,"ASCENT, 2 SCIENCE PARK DRIVE 118222",Permanent,Middle Management,"Information Technology, General Management","$12,000","$20,000",Monthly,Roles & ResponsibilitiesMedical Devices & Diag...,https://www.mycareersfuture.sg/job/director-da...
199,Migration Data Business Analyst / Data Analyst,EPAM SYSTEMS PTE. LTD.,"SPORE BUSINESS FEDERATION CTR, 160 ROBINSON RO...",Full Time,Professional,Information Technology,"$7,000","$10,500",Monthly,Roles & Responsibilities Extract and understa...,https://www.mycareersfuture.sg/job/migration-d...
1113,Manager,CHERRYLOFT FM PTE. LTD.,"CHERRYLOFT RESORTS, 159 JALAN LOYANG BESAR 509404","Permanent, Full Time","Senior Management, Manager",Hospitality,"$3,500","$6,000",Monthly,Roles & ResponsibilitiesThe Manager/Senior Man...,https://www.mycareersfuture.sg/job/manager-che...
621,ML / NLP Research Scientist,6ESTATES PTE. LTD.,NaN,Full Time,Professional,Information Technology,"$5,000","$10,000",Monthly,Roles & ResponsibilitiesWe are looking for the...,https://www.mycareersfuture.sg/job/ml-nlp-rese...


In [111]:
new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1171 entries, 0 to 1170
Data columns (total 11 columns):
job_title      1171 non-null object
company        1171 non-null object
address        896 non-null object
job_type       1171 non-null object
seniority      1115 non-null object
categories     1171 non-null object
salary_from    1030 non-null object
salary_to      1030 non-null object
salary_type    1030 non-null object
description    1171 non-null object
url            1171 non-null object
dtypes: object(11)
memory usage: 100.7+ KB


In [3]:
driver = webdriver.Chrome(chromedriver) 
driver.get('https://www.mycareersfuture.sg/search?search=data%20analytics&sortBy=new_posting_date&page=17')

In [26]:
button = driver.find_elements_by_class_name('pointer')[-2]
#button.text == '❯'
if button.text == '❯':
    button.click()
else:
    print('End of search results.')

End of search results.
